# Utils

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip freeze > req.txt

In [ ]:
!pip install scikit-learn==0.20.3 xgboost==0.82 scipy==1.4.1
!pip install speechpy
!pip install SpeechRecognition

In [ ]:
!pip show numpy

Name: numpy
Version: 1.21.6
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: None
License: BSD
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: yellowbrick, xgboost, xarray, wordcloud, torchvision, torchtext, tifffile, thinc, Theano-PyMC, tensorflow, tensorflow-probability, tensorflow-hub, tensorflow-datasets, tensorboard, tables, statsmodels, speechpy, spacy, sklearn-pandas, seaborn, scs, scipy, scikit-learn, scikit-image, resampy, qdldl, PyWavelets, python-louvain, pystan, pysndfile, pymc3, pyerfa, pyemd, pycocotools, pyarrow, plotnine, patsy, pandas, osqp, opt-einsum, opencv-python, opencv-contrib-python, numexpr, numba, nibabel, netCDF4, moviepy, mlxtend, mizani, missingno, matplotlib, matplotlib-venn, lightgbm, librosa, Keras-Preprocessing, kapre, jpeg4py, jaxlib, jax, imgaug, imbalanced-learn, imageio, hyperopt, holoviews, h5py, gym, gensim, folium, f

In [ ]:
from google.colab import drive

import speech_recognition as speechrec
import librosa # to extract speech features
import librosa.display
from IPython.display import Audio
import soundfile # to read audio file
import scipy.io.wavfile as wav
from speechpy.feature import mfcc

from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd

import math
import re
import glob
import os
import sys
from tqdm import tqdm
from typing import Tuple

import sklearn
from sklearn.model_selection import train_test_split # for splitting training and testing
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model
from sklearn.metrics import accuracy_score # to measure how good we are
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC, LinearSVC
from keras import Sequential
from keras.layers import LSTM as KERAS_LSTM, Dense, Dropout, Conv2D, Flatten, \
    BatchNormalization, Activation, MaxPooling2D
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import load_model

import pickle # to save model after training
import tensorflow as tf

# import soundfile as sf

In [ ]:
# !mkdir /content/wav/
!mkdir /content/pre-processed/
!mkdir /content/transcriptions/

mkdir: cannot create directory ‘/content/pre-processed/’: File exists
mkdir: cannot create directory ‘/content/transcriptions/’: File exists


# ENV Variables

In [ ]:
MODEL_STORAGE_ML = "/content/gdrive/MyDrive/Major Project Sem 7 8/ser3/17042022_sess(1,2,3)/" #Stores all the saved pickle files for all the models
TRANSCRIPTION_PATH = '/content/transcriptions/'
WAV_FILE_PATH = '/content/test/' #directory path containing all test wav files
DATA_DIR = '/content/pre-processed/' #directory to store all the preprocessed audio vectors
LSTM_PATH ='gdrive/MyDrive/Major Project Sem 7 8/ser2/20220301_071644/' #lstm saved model in google drive
CNN_2_PATH = 'gdrive/MyDrive/Major Project Sem 7 8/ser2/20220301_075730/' #cnn saved model in goofle drive (abstract class)
NLP_CNN_PATH = 'gdrive/MyDrive/Major Project Sem 7 8/NLP/CNN/'
max_seq_len = 500
NLP_WEIGHTAGE = 0.4
SER_WEIGHTAGE = 0.6

In [ ]:
drive.mount('/content/gdrive')
filereader = open('gdrive/MyDrive/Major Project Sem 7 8/buzzword.txt','r')
buzzwords = filereader.read().splitlines()
# buzzwords

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# !cp -a "/content/gdrive/MyDrive/Major Project Sem 7 8/wav" /content/
!cp -a "/content/gdrive/MyDrive/Major Project Sem 7 8/test" /content/

# Audio Feature Extraction and Transcription

In [ ]:
sr = 44100
audio_vectors = {}
text_df = pd.DataFrame(columns=['wav_file','text', 'text_emotion_hap', 'text_emotion_ang', 'text_emotion_sad', 'text_emotion_neu','buzzword_present'])

df_testaudios= pd.DataFrame(columns=['wav_file','emotion']) #Dataframe to store the names of all the wav files and corresponding predicted emotion

orig_wav_filenames = []

orig_wav_files = os.listdir(WAV_FILE_PATH) #lists all the wav files in the directory

"""Creating all the audio vectors from the wav files using librosa library"""

for orig_wav_file in tqdm(orig_wav_files):
    try:
        orig_wav_vector, _sr = librosa.load(WAV_FILE_PATH + orig_wav_file, sr=sr)
        orig_wav_file, file_format = orig_wav_file.split('.')
        truncated_wav_vector = orig_wav_vector[0:]
        audio_vectors[orig_wav_file] = truncated_wav_vector
        orig_wav_filenames.append(orig_wav_file)
    except:
        print('An exception occured for {}'.format(orig_wav_file))

df_testaudios['wav_file'] = orig_wav_filenames #Storing the names of all the test wav files


with open('/content/pre-processed/audio_vectors_test.pkl', 'wb') as f:   #Storing all the audio vectors
    pickle.dump(audio_vectors, f)



"""Extracting features from audio vectors - 8 features as shown by column headers"""

# audio_vectors_path = '{}audio_vectors_test.pkl'.format(DATA_DIR)
# audio_vectors = pickle.load(open(audio_vectors_path, 'rb'))

columns = ['wav_file', 'sig_mean', 'sig_std', 'rmse_mean', 'rmse_std', 'silence', 'harmonic', 'auto_corr_max', 'auto_corr_std','ser_hap','ser_ang','ser_sad','ser_neu'] #columns represent all the features extracted added to dataframe

df_features = pd.DataFrame(columns=columns)

# print(df_testaudios.head())
# print(df_testaudios['wav_file'].head())
r = speechrec.Recognizer()

print(df_testaudios)

for index, row in tqdm(df_testaudios['wav_file'].to_frame().iterrows()):

  try:
    wav_file_name = row['wav_file']
    y = audio_vectors[wav_file_name]

    with speechrec.AudioFile(os.path.join(WAV_FILE_PATH,wav_file_name+".wav")) as source:
      audio_text = r.listen(source)
      text = r.recognize_google(audio_text)
      # print(text)
      # with open(os.path.join(TRANSCRIPTION_PATH,wav_file_name+".txt"), "w") as file1:
      #   # Writing data to a file
      #   file1.write(text)

      new_row = {'wav_file':wav_file_name, 'text':text, 'text_emotion_hap':None,'text_emotion_ang':None,'text_emotion_sad':None,'text_emotion_neu':None}
      text_df = text_df.append(new_row, ignore_index=True)
        # file1.close()

    feature_list = [wav_file_name]  # wav_file
    sig_mean = np.mean(abs(y))
    feature_list.append(sig_mean)  # sig_mean
    feature_list.append(np.std(y))  # sig_std

    rmse = librosa.feature.rms(y + 0.0001)[0]
    feature_list.append(np.mean(rmse))  # rmse_mean
    feature_list.append(np.std(rmse))  # rmse_std

    silence = 0
    for e in rmse:
      if e <= 0.4 * np.mean(rmse):
        silence += 1
    silence /= float(len(rmse))
    feature_list.append(silence)  # silence

    y_harmonic = librosa.effects.hpss(y)[0]
    feature_list.append(np.mean(y_harmonic) * 1000)  # harmonic (scaled by 1000)


    cl = 0.45 * sig_mean
    center_clipped = []
    for s in y:
      if s >= cl:
        center_clipped.append(s - cl)
      elif s <= -cl:
        center_clipped.append(s + cl)
      elif np.abs(s) < cl:
        center_clipped.append(0)
    auto_corrs = librosa.core.autocorrelate(np.array(center_clipped))
    feature_list.append(1000 * np.max(auto_corrs)/len(auto_corrs))  # auto_corr_max (scaled by 1000)
    feature_list.append(np.std(auto_corrs))  # auto_corr_std
    feature_list.extend([0,0,0,0])
    df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)
  except:
    print('Some exception occured')

df = df_features
scalar = MinMaxScaler()
df[df.columns[2:]] = scalar.fit_transform(df[df.columns[2:]])

df.to_csv('{}/audio_features.csv'.format(DATA_DIR), index=False) #storing all the features in a csv
text_df.to_csv('{}/audio_transcript.csv'.format(TRANSCRIPTION_PATH), index=False)

100%|██████████| 23/23 [00:05<00:00,  4.06it/s]


               wav_file emotion
0   Ses01F_impro01_M006     NaN
1   Ses01F_impro01_F013     NaN
2   Ses01F_impro01_F012     NaN
3   Ses01F_impro01_M008     NaN
4   Ses01F_impro01_M000     NaN
5   Ses01F_impro01_F006     NaN
6   Ses01F_impro02_M008     NaN
7   Ses01F_impro01_M003     NaN
8   Ses01F_impro01_F015     NaN
9   Ses01F_impro01_M004     NaN
10  Ses01F_impro01_M010     NaN
11  Ses01F_impro01_F008     NaN
12  Ses01F_impro01_M001     NaN
13  Ses01F_impro01_M007     NaN
14  Ses01F_impro01_F009     NaN
15  Ses01F_impro01_M011     NaN
16  Ses01F_impro02_M003     NaN
17  Ses01F_impro01_F011     NaN
18  Ses01F_impro01_F007     NaN
19  Ses01F_impro01_M002     NaN
20  Ses01F_impro01_M005     NaN
21  Ses01F_impro01_M009     NaN
22  Ses01F_impro01_M013     NaN


7it [00:27,  2.97s/it]

Some exception occured


23it [01:20,  3.52s/it]
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [ ]:
x_test = pd.read_csv('{}/audio_features.csv'.format(DATA_DIR))
# del x_test['wav_file'] #dropping the name of the file to keep only the features for prediction

In [ ]:
# x_test.columns


In [ ]:
x_testset = x_test.drop(['wav_file','ser_hap','ser_sad','ser_ang','ser_neu'], axis = 1)

In [ ]:
emotion_dict = {0:"Angry", 1:"Happy", 2:"Sad", 3:"Neutral"}

In [ ]:
# x_test.columns

# Random Forest - SER

In [ ]:
rf_classifier = pickle.load(open('{}rf_classifier.model'.format(MODEL_STORAGE_ML), 'rb'))
pred = rf_classifier.predict_proba(x_testset)
# print(pred)
for idx, val in enumerate(pred):
    # print("\nPrediction for the file {} is: \t{}".format(df['wav_file'][idx],emotion_dict[val]))
    x_test.iat[idx,9]=val[1]
    x_test.iat[idx,10]=val[0]
    x_test.iat[idx,11]=val[2]
    x_test.iat[idx,12]=val[3]

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)


In [ ]:
# x_test

## NLP

In [ ]:
tokenizer = Tokenizer()
with open('gdrive/MyDrive/Major Project Sem 7 8/NLP/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
def clean_text(data):

    # remove hashtags and @usernames
    data = re.sub(r"(#[\d\w\.]+)", '', data)
    data = re.sub(r"(@[\d\w\.]+)", '', data)

    # tekenization using nltk
    data = word_tokenize(data)

    return data

In [ ]:
nlp_model=pickle.load(open('/content/gdrive/MyDrive/Major Project Sem 7 8/NLP/cnn.model', 'rb'))

In [ ]:
class_names = ['hap', 'xxx', 'anger', 'sadness', 'neutral']
# X_test = text_df.drop('file_name','emotion')


for index,row in tqdm(text_df.iterrows()):
  flag = False
  text_file_name, text = row[0], row[1]
  words_re = re.compile("|".join(buzzwords))
  if words_re.search(text):
    flag = True
  # text=['I cannot believe I got an admit!']
  seq = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(seq, maxlen=max_seq_len)
  pred = nlp_model.predict(padded)
  # print(seq)
  # print(padded)
  # print(pred)
  # print(text_file_name+"\t\t"+class_names[np.argmax(pred)])
  text_df.iat[index,2] = pred[0][0]
  text_df.iat[index,3] = pred[0][2]
  text_df.iat[index,4] = pred[0][3]
  text_df.iat[index,5] = pred[0][4]
  text_df.iat[index,6] = flag

# print('\n')
# print(text_df)

22it [00:01, 15.20it/s]


In [ ]:
# combined_df = pd.merge(x_test, text_df, on="wav_file")
# combined_df
dfinal = x_test.merge(text_df, on="wav_file")
dfinal

,wav_file,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std,ser_hap,ser_ang,ser_sad,ser_neu,text,text_emotion_hap,text_emotion_ang,text_emotion_sad,text_emotion_neu,buzzword_present
0,Ses01F_impro01_M006,0.063108,0.479429,0.559604,0.404965,0.484395,0.198778,0.227262,0.524585,0.056278,0.742408,0.032459,0.168855,no I need I need you I need another's set of I...,0.212189,0.128981,0.114056,0.521779,False
1,Ses01F_impro01_F013,0.027741,0.232551,0.225787,0.239067,0.825699,0.079384,0.070422,0.050130,0.040715,0.513545,0.273073,0.172667,how long have you been working here,0.199137,0.056092,0.018577,0.724902,True
2,Ses01F_impro01_F012,0.035311,0.390844,0.311220,0.445606,0.834461,0.008658,0.190674,0.170293,0.032450,0.566187,0.301672,0.099691,that's out of control,0.008997,0.391823,0.042817,0.06662,False
3,Ses01F_impro01_M008,0.040258,0.400824,0.352526,0.436345,0.967795,0.117785,0.191323,0.200280,0.034401,0.569090,0.310636,0.085872,no I need another ID a separate one,0.162762,0.088149,0.366129,0.361223,False
4,Ses01F_impro01_M000,0.010217,0.068525,0.057622,0.076066,0.879329,0.000000,0.011046,0.018572,0.067025,0.523154,0.108477,0.301344,give your forms,0.201602,0.014537,0.004356,0.772113,False
5,Ses01F_impro01_F006,0.062079,0.497531,0.560515,0.441102,0.347313,0.067685,0.253289,0.211195,0.116804,0.719484,0.088058,0.075654,what I'm getting an ID this is why I'm here my...,0.004801,0.888898,0.087623,0.005649,False
6,Ses01F_impro01_M003,0.041165,0.306897,0.359980,0.258414,0.548509,0.075620,0.103924,0.117645,0.063400,0.573511,0.274296,0.088792,okay but I didn't tell you to get in this line...,0.160922,0.235626,0.248818,0.23924,False
7,Ses01F_impro01_F015,0.085525,0.646402,0.791714,0.500679,0.159223,0.074439,0.389942,0.202040,0.111748,0.717278,0.083721,0.087253,I'd appreciate that yeah,0.255041,0.0095,0.007329,0.717568,False
8,Ses01F_impro01_M004,0.057450,0.459125,0.508465,0.415682,0.583281,0.135290,0.217527,0.247082,0.055009,0.548997,0.309401,0.086592,this form is a zx4,0.37552,0.019639,0.017148,0.562999,False
9,Ses01F_impro01_M010,0.048448,0.362752,0.436739,0.292242,0.481667,0.112626,0.138084,0.119985,0.101290,0.618105,0.180476,0.100129,yeah we keep it on file but we need an ID to a...,0.032628,0.041674,0.071348,0.821357,False


In [ ]:
dfinal['combined_score_hap'] = dfinal['ser_hap']*SER_WEIGHTAGE + dfinal['text_emotion_hap']*NLP_WEIGHTAGE
dfinal['combined_score_sad'] = dfinal['ser_sad']*SER_WEIGHTAGE + dfinal['text_emotion_sad']*NLP_WEIGHTAGE
dfinal['combined_score_ang'] = dfinal['ser_ang']*SER_WEIGHTAGE + dfinal['text_emotion_ang']*NLP_WEIGHTAGE
dfinal['combined_score_neu'] = dfinal['ser_neu']*SER_WEIGHTAGE+ dfinal['text_emotion_neu']*NLP_WEIGHTAGE
# dfinal

In [ ]:
dfinal['combined_score_neu'] = pd.to_numeric(dfinal['combined_score_neu'],errors = 'coerce')
dfinal['combined_score_sad'] = pd.to_numeric(dfinal['combined_score_sad'],errors = 'coerce')
dfinal['combined_score_hap'] = pd.to_numeric(dfinal['combined_score_hap'],errors = 'coerce')
dfinal['combined_score_ang'] = pd.to_numeric(dfinal['combined_score_ang'],errors = 'coerce')

In [ ]:
dfinal['predicted_emotion'] = dfinal.iloc[:,-4:].idxmax(axis=1)
dfinal['predicted_emotion'] = dfinal['predicted_emotion'].str.slice(-3)

In [ ]:
dfinal['predicted_proba'] = dfinal.iloc[:,-4:].max(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [ ]:
dfinal

,wav_file,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std,ser_hap,...,text_emotion_ang,text_emotion_sad,text_emotion_neu,buzzword_present,combined_score_hap,combined_score_sad,combined_score_ang,combined_score_neu,predicted_emotion,predicted_proba
0,Ses01F_impro01_M006,0.063108,0.479429,0.559604,0.404965,0.484395,0.198778,0.227262,0.524585,0.056278,...,0.128981,0.114056,0.521779,False,0.118642,0.065098,0.497037,0.310025,ang,0.497037
1,Ses01F_impro01_F013,0.027741,0.232551,0.225787,0.239067,0.825699,0.079384,0.070422,0.050130,0.040715,...,0.056092,0.018577,0.724902,True,0.104084,0.171275,0.330564,0.393561,neu,0.393561
2,Ses01F_impro01_F012,0.035311,0.390844,0.311220,0.445606,0.834461,0.008658,0.190674,0.170293,0.032450,...,0.391823,0.042817,0.06662,False,0.023069,0.198130,0.496442,0.086463,ang,0.496442
3,Ses01F_impro01_M008,0.040258,0.400824,0.352526,0.436345,0.967795,0.117785,0.191323,0.200280,0.034401,...,0.088149,0.366129,0.361223,False,0.085745,0.332833,0.376714,0.196013,ang,0.376714
4,Ses01F_impro01_M000,0.010217,0.068525,0.057622,0.076066,0.879329,0.000000,0.011046,0.018572,0.067025,...,0.014537,0.004356,0.772113,False,0.120856,0.066829,0.319707,0.489652,neu,0.489652
5,Ses01F_impro01_F006,0.062079,0.497531,0.560515,0.441102,0.347313,0.067685,0.253289,0.211195,0.116804,...,0.888898,0.087623,0.005649,False,0.072003,0.087884,0.787250,0.047652,ang,0.787250
6,Ses01F_impro01_M003,0.041165,0.306897,0.359980,0.258414,0.548509,0.075620,0.103924,0.117645,0.063400,...,0.235626,0.248818,0.23924,False,0.102409,0.264105,0.438357,0.148971,ang,0.438357
7,Ses01F_impro01_F015,0.085525,0.646402,0.791714,0.500679,0.159223,0.074439,0.389942,0.202040,0.111748,...,0.0095,0.007329,0.717568,False,0.169065,0.053164,0.434167,0.339379,ang,0.434167
8,Ses01F_impro01_M004,0.057450,0.459125,0.508465,0.415682,0.583281,0.135290,0.217527,0.247082,0.055009,...,0.019639,0.017148,0.562999,False,0.183213,0.192500,0.337254,0.277155,ang,0.337254
9,Ses01F_impro01_M010,0.048448,0.362752,0.436739,0.292242,0.481667,0.112626,0.138084,0.119985,0.101290,...,0.041674,0.071348,0.821357,False,0.073825,0.136825,0.387533,0.388621,neu,0.388621


In [ ]:
labels_df = pd.read_csv('/content/df_iemocap.csv')
dfinal = dfinal.merge(labels_df, on="wav_file",how='left')

In [ ]:
len(labels_df)

5766

In [ ]:
dfinal

,wav_file,sig_mean,sig_std,rmse_mean,rmse_std,silence,harmonic,auto_corr_max,auto_corr_std,ser_hap,...,combined_score_ang,combined_score_neu,predicted_emotion,predicted_proba,start_time,end_time,emotion,val,act,dom
0,Ses01F_impro01_M006,0.063108,0.479429,0.559604,0.404965,0.484395,0.198778,0.227262,0.524585,0.056278,...,0.497037,0.310025,ang,0.497037,41.2300,46.9800,ang,2.0,3.5,3.5
1,Ses01F_impro01_F013,0.027741,0.232551,0.225787,0.239067,0.825699,0.079384,0.070422,0.050130,0.040715,...,0.330564,0.393561,neu,0.393561,93.6700,97.0218,ang,2.0,4.0,3.5
2,Ses01F_impro01_F012,0.035311,0.390844,0.311220,0.445606,0.834461,0.008658,0.190674,0.170293,0.032450,...,0.496442,0.086463,ang,0.496442,85.2700,88.0200,ang,2.0,3.5,3.5
3,Ses01F_impro01_M008,0.040258,0.400824,0.352526,0.436345,0.967795,0.117785,0.191323,0.200280,0.034401,...,0.376714,0.196013,ang,0.376714,58.1800,62.5900,ang,2.0,3.5,3.5
4,Ses01F_impro01_M000,0.010217,0.068525,0.057622,0.076066,0.879329,0.000000,0.011046,0.018572,0.067025,...,0.319707,0.489652,neu,0.489652,7.5712,10.4750,ang,2.5,2.0,2.5
5,Ses01F_impro01_F006,0.062079,0.497531,0.560515,0.441102,0.347313,0.067685,0.253289,0.211195,0.116804,...,0.787250,0.047652,ang,0.787250,38.9650,43.5900,ang,2.0,3.5,3.5
6,Ses01F_impro01_M003,0.041165,0.306897,0.359980,0.258414,0.548509,0.075620,0.103924,0.117645,0.063400,...,0.438357,0.148971,ang,0.438357,23.4700,28.0300,ang,2.5,3.5,3.5
7,Ses01F_impro01_F015,0.085525,0.646402,0.791714,0.500679,0.159223,0.074439,0.389942,0.202040,0.111748,...,0.434167,0.339379,ang,0.434167,103.9700,106.7100,ang,2.0,3.5,3.0
8,Ses01F_impro01_M004,0.057450,0.459125,0.508465,0.415682,0.583281,0.135290,0.217527,0.247082,0.055009,...,0.337254,0.277155,ang,0.337254,28.3950,31.2117,ang,2.0,4.0,3.5
9,Ses01F_impro01_M010,0.048448,0.362752,0.436739,0.292242,0.481667,0.112626,0.138084,0.119985,0.101290,...,0.387533,0.388621,neu,0.388621,81.5900,86.0300,ang,2.0,4.5,4.0


In [ ]:
from sklearn.metrics import accuracy_score
acc_score = accuracy_score(dfinal['emotion'],dfinal['predicted_emotion'])

In [ ]:
acc_score

0.6818181818181818

In [ ]:
"""
Emotion [Angry-8 , Happiness-3 , Sadness-5 , Neutral-1 ]
Emotion Score [predicted probability]
Buzzword Score[1]
Loyalty Score[prime-2,normal-1,not a customer-0]
Disconnet&Call back score[+]
WaitingTime score[linear*1]

"""

'\nEmotion [Angry-8 , Happiness-3 , Sadness-5 , Neutral-1 ]\nEmotion Score [predicted probability]\nBuzzword Score[1]\nLoyalty Score[prime-2,normal-1,not a customer-0]\nDisconnet&Call back score[+]\nWaitingTime score[linear*1]\n\n'

In [ ]:
waiting_queue = []

In [ ]:
# angry:1, happy:2, sad:3, netural:4, 0:none

from inspect import _void
import pandas as pd
import numpy as np
from datetime import datetime
import random
import time

now = datetime.now()

uid = 0
emo_factor =0
n=3
agents = [[1, 2, 3, 4],
          [2, 3, 4, 1],
          [1, 4, 0, 0],
          [1, 2, 3, 4],
          [4, 2, 3, 0],
          [2, 1, 0, 0],
          [1, 2, 3, 4],
          [1, 0, 0, 0],
          [2, 4, 3, 1],
          [1, 2, 3, 4]]


users = []

df=dfinal
# df = pd.read_csv('audio_features.csv')
for index,row in df.iterrows():
    flag = 0
    user = dict()
    user['user_id'] = uid
    user['wav_file_name'] = row['wav_file']
    user['emotion'] = row['predicted_emotion']
    user['loyalty'] = random.randint(0,2) #loyalty randomly generated value in (0,1,2) indicating membership level
    user['callback'] = random.randint(0,1) #callback indicated if the person has called back in last 30 mins after disconnecting once
    user['arrival_time'] = time.time()
    if(row['buzzword_present']==True):
        flag = 1
    if(user['emotion']=='ang'):
        emo_factor=8
    elif(user['emotion']=='sad'):
        emo_factor=5
    elif(user['emotion']=='hap'):
        emo_factor=3
    elif(user['emotion']=='neu'):
        emo_factor=1
    user['score'] = row['predicted_proba']*emo_factor + flag + user['callback'] + user['loyalty']
    user['temp_score'] = 0
    users.append(user)
    uid+=1


current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

def updateQueue(d,waiting_queue,rearrange=False):
    if(rearrange==False):
      for key in d:
          print("Key",key,"Value",d[key])
      waiting_queue.append(d)

    for user in waiting_queue:
        user['temp_score']= user['score']+(time.time()-user['arrival_time'])/60
    waiting_queue = sorted(waiting_queue, key=lambda dic: dic['temp_score'], reverse=True)
    print(waiting_queue)
    return waiting_queue

Current Time = 04:26:26


In [ ]:
users

[{'arrival_time': 1650515186.1424904,
  'callback': 0,
  'emotion': 'ang',
  'loyalty': 1,
  'score': 4.9715340438310776,
  'temp_score': 0,
  'user_id': 0,
  'wav_file_name': 'Ses01F_impro01_F012'},
 {'arrival_time': 1650515186.142596,
  'callback': 1,
  'emotion': 'hap',
  'loyalty': 2,
  'score': 3.972476087091655,
  'temp_score': 0,
  'user_id': 1,
  'wav_file_name': 'Ses01F_impro01_M009'},
 {'arrival_time': 1650515186.1426733,
  'callback': 0,
  'emotion': 'ang',
  'loyalty': 0,
  'score': 3.991907347138988,
  'temp_score': 0,
  'user_id': 2,
  'wav_file_name': 'Ses01F_impro01_M005'},
 {'arrival_time': 1650515186.1427445,
  'callback': 1,
  'emotion': 'ang',
  'loyalty': 1,
  'score': 5.473336682822515,
  'temp_score': 0,
  'user_id': 3,
  'wav_file_name': 'Ses01F_impro01_F015'},
 {'arrival_time': 1650515186.1428142,
  'callback': 1,
  'emotion': 'ang',
  'loyalty': 2,
  'score': 8.646235853718775,
  'temp_score': 0,
  'user_id': 4,
  'wav_file_name': 'Ses01F_impro01_F011'},
 {'ar

In [ ]:
updateQueue({'arrival_time': time.time(),
  'callback': 1,
  'emotion': 'hap',
  'loyalty': 1,
  'score': 10,
  'temp_score': 0,
  'user_id': 3,
  'wav_file_name': 'this is trial'},waiting_queue)

for u in users:
  updateQueue(u,waiting_queue)

Key arrival_time Value 1650515186.1728246
Key callback Value 1
Key emotion Value hap
Key loyalty Value 1
Key score Value 10
Key temp_score Value 0
Key user_id Value 3
Key wav_file_name Value this is trial
[{'arrival_time': 1650515186.1728246, 'callback': 1, 'emotion': 'hap', 'loyalty': 1, 'score': 10, 'temp_score': 10.000104574362437, 'user_id': 3, 'wav_file_name': 'this is trial'}]
Key user_id Value 0
Key wav_file_name Value Ses01F_impro01_F012
Key emotion Value ang
Key loyalty Value 1
Key callback Value 0
Key arrival_time Value 1650515186.1424904
Key score Value 4.9715340438310776
Key temp_score Value 0
[{'arrival_time': 1650515186.1728246, 'callback': 1, 'emotion': 'hap', 'loyalty': 1, 'score': 10, 'temp_score': 10.000121374924978, 'user_id': 3, 'wav_file_name': 'this is trial'}, {'user_id': 0, 'wav_file_name': 'Ses01F_impro01_F012', 'emotion': 'ang', 'loyalty': 1, 'callback': 0, 'arrival_time': 1650515186.1424904, 'score': 4.9715340438310776, 'temp_score': 4.972161021115827}]
Key u

In [ ]:
updateQueue({'arrival_time': time.time(),
  'callback': 0,
  'emotion': 'ang',
  'loyalty': 1,
  'score': 5,
  'temp_score': 0,
  'user_id': 4,
  'wav_file_name': 'this is trial2'},waiting_queue)

Key arrival_time Value 1650515186.263456
Key callback Value 0
Key emotion Value ang
Key loyalty Value 1
Key score Value 5
Key temp_score Value 0
Key user_id Value 4
Key wav_file_name Value this is trial2
[{'arrival_time': 1650515186.1728246, 'callback': 1, 'emotion': 'hap', 'loyalty': 1, 'score': 10, 'temp_score': 10.001584577560426, 'user_id': 3, 'wav_file_name': 'this is trial'}, {'user_id': 4, 'wav_file_name': 'Ses01F_impro01_F011', 'emotion': 'ang', 'loyalty': 2, 'callback': 1, 'arrival_time': 1650515186.1428142, 'score': 8.646235853718775, 'temp_score': 8.648320720878301}, {'user_id': 10, 'wav_file_name': 'Ses01F_impro01_M007', 'emotion': 'ang', 'loyalty': 2, 'callback': 1, 'arrival_time': 1650515186.1432161, 'score': 7.330374421413872, 'temp_score': 7.33245267643147}, {'user_id': 21, 'wav_file_name': 'Ses01F_impro01_F006', 'emotion': 'ang', 'loyalty': 0, 'callback': 1, 'arrival_time': 1650515186.1439705, 'score': 7.297998508091252, 'temp_score': 7.300064432894668}, {'user_id': 12

[{'arrival_time': 1650515186.1728246,
  'callback': 1,
  'emotion': 'hap',
  'loyalty': 1,
  'score': 10,
  'temp_score': 10.001584577560426,
  'user_id': 3,
  'wav_file_name': 'this is trial'},
 {'arrival_time': 1650515186.1428142,
  'callback': 1,
  'emotion': 'ang',
  'loyalty': 2,
  'score': 8.646235853718775,
  'temp_score': 8.648320720878301,
  'user_id': 4,
  'wav_file_name': 'Ses01F_impro01_F011'},
 {'arrival_time': 1650515186.1432161,
  'callback': 1,
  'emotion': 'ang',
  'loyalty': 2,
  'score': 7.330374421413872,
  'temp_score': 7.33245267643147,
  'user_id': 10,
  'wav_file_name': 'Ses01F_impro01_M007'},
 {'arrival_time': 1650515186.1439705,
  'callback': 1,
  'emotion': 'ang',
  'loyalty': 0,
  'score': 7.297998508091252,
  'temp_score': 7.300064432894668,
  'user_id': 21,
  'wav_file_name': 'Ses01F_impro01_F006'},
 {'arrival_time': 1650515186.1433492,
  'callback': 1,
  'emotion': 'ang',
  'loyalty': 1,
  'score': 7.0891564486430205,
  'temp_score': 7.091232510209691,
  

In [ ]:
agent_list=[]
agent_waiting_queue=dict.fromkeys(list(range(n)))
emotion_matrix = []

def init_agents(n):
    global agent_list
    global emotion_matrix
    for i in range(n-1):
        agent=dict()
        agent['agent_id']=i
        agent['preference']=[]
        from random import randrange
        zeros=randrange(4)
        agent['preference'] = random.sample(range(1,5), 4-zeros)
        for j in range(zeros):
            agent['preference'].append(0)
        agent_list.append(agent)
        emotion_matrix.append(agent['preference'])
    agent_list.append({'agent_id': n-1, 'preference': [4, 3, 1, 2]})
    emotion_matrix.append([4, 3, 1, 2])
    emotion_matrix = np.array(emotion_matrix)


In [ ]:
# agent_waiting_queue=dict.fromkeys(list(range(n)))

In [ ]:
init_agents(3)

In [ ]:
agent_list

[{'agent_id': 0, 'preference': [1, 4, 3, 2]},
 {'agent_id': 1, 'preference': [4, 0, 0, 0]},
 {'agent_id': 2, 'preference': [4, 3, 1, 2]}]

In [ ]:
encoding = {
    'ang':1,
    'hap':2,
    'sad':3,
    'neu':4
}
def agent_suitability(n):
    global emotion_matrix
    global waiting_queue
    global agent_waiting_queue
    for user in waiting_queue:
        agent_suitability_list = []
        asl = []
        for i in range(4):
            for j in range(n):
                if(encoding[user['emotion']]==emotion_matrix[j][i]):
                    agent_suitability_list.append(agent_list[j])
                    asl.append(agent_list[j]['agent_id'])
                    user_temp = user.copy()
                    user_temp['score'] = user_temp['score']*(1-(len(asl)-1)/n)
                    try:
                      agent_waiting_queue[agent_list[j]['agent_id']].append(user_temp)
                    except:
                      agent_waiting_queue[agent_list[j]['agent_id']]= []
                      agent_waiting_queue[agent_list[j]['agent_id']].append(user_temp)

        # print(agent_suitability_list)
        for agent in agent_list:
          if(agent['agent_id'] in asl):
            continue
          else:
            user_temp = user.copy()
            user_temp['score'] = user_temp['score']*(1-len(asl)/n)
            try:
              agent_waiting_queue[agent['agent_id']].append(user_temp)
            except:
              agent_waiting_queue[agent['agent_id']] = []
              agent_waiting_queue[agent['agent_id']].append(user_temp)
        print(user['user_id'],"\t",agent_suitability_list,"\n\n")


In [ ]:
agent_suitability(3)

3 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


0 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


1 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


2 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


3 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


4 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


5 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


6 	 [{'agent_id': 0, 'preference': [1, 4, 3, 2]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}] 


7 	 [{'agent_id': 1, 'preference': [4, 0, 0, 0]}, {'agent_id': 2, 'preference': [4, 3, 1, 2]}, {'agent_id': 0, 'preference': [1, 4, 3, 2]}] 


8 	 [{'agent_id': 1, 'preference': [4, 0, 0, 0]}, {'agent_id': 2, 'prefer

In [ ]:
agent_waiting_queue

{0: [{'arrival_time': 1650515186.1728246,
   'callback': 1,
   'emotion': 'hap',
   'loyalty': 1,
   'score': 10.0,
   'temp_score': 10.001584577560426,
   'user_id': 3,
   'wav_file_name': 'this is trial'},
  {'arrival_time': 1650515186.1424904,
   'callback': 0,
   'emotion': 'ang',
   'loyalty': 1,
   'score': 4.9715340438310776,
   'temp_score': 4.973624243619489,
   'user_id': 0,
   'wav_file_name': 'Ses01F_impro01_F012'},
  {'arrival_time': 1650515186.142596,
   'callback': 1,
   'emotion': 'hap',
   'loyalty': 2,
   'score': 3.972476087091655,
   'temp_score': 3.974564550398082,
   'user_id': 1,
   'wav_file_name': 'Ses01F_impro01_M009'},
  {'arrival_time': 1650515186.1426733,
   'callback': 0,
   'emotion': 'ang',
   'loyalty': 0,
   'score': 3.991907347138988,
   'temp_score': 3.993994534906017,
   'user_id': 2,
   'wav_file_name': 'Ses01F_impro01_M005'},
  {'arrival_time': 1650515186.1427445,
   'callback': 1,
   'emotion': 'ang',
   'loyalty': 1,
   'score': 5.47333668282251

In [ ]:
for key, value in agent_waiting_queue.items():
  agent_waiting_queue[key] = updateQueue({},value,rearrange=True)
  print("\n")

[{'arrival_time': 1650515186.1728246, 'callback': 1, 'emotion': 'hap', 'loyalty': 1, 'score': 10.0, 'temp_score': 10.005050031344096, 'user_id': 3, 'wav_file_name': 'this is trial'}, {'user_id': 4, 'wav_file_name': 'Ses01F_impro01_F011', 'emotion': 'ang', 'loyalty': 2, 'callback': 1, 'arrival_time': 1650515186.1428142, 'score': 8.646235853718775, 'temp_score': 8.651786170688329}, {'user_id': 10, 'wav_file_name': 'Ses01F_impro01_M007', 'emotion': 'ang', 'loyalty': 2, 'callback': 1, 'arrival_time': 1650515186.1432161, 'score': 7.330374421413872, 'temp_score': 7.33591814213607}, {'user_id': 21, 'wav_file_name': 'Ses01F_impro01_F006', 'emotion': 'ang', 'loyalty': 0, 'callback': 1, 'arrival_time': 1650515186.1439705, 'score': 7.297998508091252, 'temp_score': 7.30352984694191}, {'user_id': 12, 'wav_file_name': 'Ses01F_impro01_M002', 'emotion': 'ang', 'loyalty': 1, 'callback': 1, 'arrival_time': 1650515186.1433492, 'score': 7.0891564486430205, 'temp_score': 7.0946979918088635}, {'user_id': 16

In [ ]:
for index,row in dfinal.iterrows():
  if(row[23]=='ang'):
    print(row[0])


Ses01F_impro01_F012
Ses01F_impro01_M005
Ses01F_impro01_F015
Ses01F_impro01_F011
Ses01F_impro01_M004
Ses01F_impro01_M013
Ses01F_impro01_M007
Ses01F_impro01_M002
Ses01F_impro01_F008
Ses01F_impro01_M011
Ses01F_impro01_F007
Ses01F_impro01_M008
Ses01F_impro01_M003
Ses01F_impro01_M006
Ses01F_impro01_F006


In [ ]:
test = dict()
for index,row in dfinal.iterrows():
  if(row[23]=='ang'):
    test[row[0]] = row[24]

(sorted(test.items(), key =lambda kv:(kv[1], kv[0])))

[('Ses01F_impro01_M004', 0.3372539340062722),
 ('Ses01F_impro01_M008', 0.3767139543486031),
 ('Ses01F_impro01_M011', 0.4145683588703367),
 ('Ses01F_impro01_M007', 0.416296802676734),
 ('Ses01F_impro01_F015', 0.4341670853528143),
 ('Ses01F_impro01_M003', 0.43835696129303187),
 ('Ses01F_impro01_F008', 0.4461820711374959),
 ('Ses01F_impro01_M013', 0.4691446076489295),
 ('Ses01F_impro01_F012', 0.4964417554788847),
 ('Ses01F_impro01_M006', 0.4970372611208194),
 ('Ses01F_impro01_M005', 0.4989884183923735),
 ('Ses01F_impro01_F007', 0.5815077436268754),
 ('Ses01F_impro01_M002', 0.6361445560803776),
 ('Ses01F_impro01_F011', 0.705779481714847),
 ('Ses01F_impro01_F006', 0.7872498135114065)]